# Import Packages

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [9]:
input_df = pd.read_csv("./data/20240117_churn_data.csv", low_memory=False)

display(input_df[input_df["policy_nr_hashed"] == "dBy9zk9"])

,policy_nr_hashed,years_since_policy_started,year_initiation_policy,year_initiation_policy_version,year_end_policy,d_churn,d_churn_cancellation,d_churn_between_prolongations,d_churn_around_prolongation,premium_main_coverages,...,premium_change_mutation_8,mutation_9,premium_change_mutation_9,mutation_10,premium_change_mutation_10,mutation_11,premium_change_mutation_11,mutation_12,premium_change_mutation_12,data_collection_date
386441,dBy9zk9,0,2022,2022.0,2023.0,0,0,0,0,4919.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15
437424,dBy9zk9,1,2022,2023.0,2023.0,1,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-15
